## Using Code Interpreter with Responses API

### Environment Setup

In [1]:
# Import required packages
import os
import json
import requests
from pathlib import Path
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [2]:
# Set up environment variables for Azure OpenAI
AOAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")

### Azure OpenAI Client Setup

In [3]:
# Initialise token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

In [4]:
# Initialise Azure OpenAI client
client = AzureOpenAI(  
    azure_endpoint = AOAI_API_BASE,
    azure_ad_token_provider = token_provider,
    api_version = AOAI_API_VERSION,
)

### Testing Responses API

In [5]:
# Test Responses API
response = client.responses.create(
    model = AOAI_DEPLOYMENT,
    input = "Generate a haiku about the sea", 
)

print(response.output[0].content[0].text) 

Endless waves embrace,  
Whispering ancient secrets,  
Ocean’s heart beats calm.


### Code Interpreter output

In [6]:
# Set input prompt
INPUT_TEXT = "Could you please analyse the operating profit of Contoso Inc. using the following data and producing a bar chart image. Contoso Inc. Operating Profit: Quarter 1: $2.2 million, Quarter 2: $2.5 million, Quarter 3: $2.3 million, Quarter 4: $3.8 million, Industry Average: $2.5 million. When quarter values in 2025 fall below the industry average, please highlight them in red, otherwise they should be green."

In [7]:
# Generate a file with the code interpreter
response = client.responses.create(
    model = AOAI_DEPLOYMENT,
    tools = [
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    instructions = "You are a helpful data analyst. You should use Python tool to perform required calculations.",
    input = INPUT_TEXT
)

# print(response.model_dump_json(indent=2))

In [8]:
# Extract details of generated files
extracted_files = []

for item in response.output:
    if hasattr(item, 'content'):
        for content_block in item.content:
            if hasattr(content_block, 'annotations') and content_block.annotations:
                for annotation in content_block.annotations:
                    if hasattr(annotation, 'file_id'):
                        file_info = {
                            "container_id": getattr(annotation, 'container_id', None),
                            "file_id": getattr(annotation, 'file_id', None),
                            "filename": getattr(annotation, 'filename', None)
                        }
                        extracted_files.append(file_info)

print(json.dumps(extracted_files, indent=2))

[
  {
    "container_id": "cntr_68990aaa13b88190810d4b9a5e1cc7b20727e1a613c30bf2",
    "file_id": "cfile_68990aafbcd88190918923dc947b587b",
    "filename": "cfile_68990aafbcd88190918923dc947b587b.png"
  },
  {
    "container_id": "cntr_68990aaa13b88190810d4b9a5e1cc7b20727e1a613c30bf2",
    "file_id": "cfile_68990ab2226c81909c4595c7e54b0364",
    "filename": "contoso_operating_profit.png"
  }
]


### Troubleshooting Details

In [9]:
# Detailed output for PG
def demonstrate_container_file_access_issue():
    """
    Demonstrates the container file access issue using existing response object
    """
    
    print("🔍 Azure OpenAI - Issue with access to file in Code Interpreter's container")
    print("=" * 60)
    
    # Step 1: Extract file info from existing response
    print(f"\n1️⃣ Using existing response: {response.id}")
    
    container_id = None
    file_id = None
    filename = None
    
    for output_item in response.output:
        if hasattr(output_item, 'content'):
            for content_block in output_item.content:
                if hasattr(content_block, 'annotations') and content_block.annotations:
                    for annotation in content_block.annotations:
                        if hasattr(annotation, 'file_id'):
                            container_id = getattr(annotation, 'container_id', None)
                            file_id = getattr(annotation, 'file_id', None) 
                            filename = getattr(annotation, 'filename', None)
                            break
    
    if not file_id:
        print("   ❌ No files found in annotations")
        return False
    
    print(f"   ✅ File found in annotations:")
    print(f"      Container ID: {container_id}")
    print(f"      File ID: {file_id}")
    print(f"      Filename: {filename}")
    
    # Step 2: Attempt download using OpenAI documented pattern
    print(f"\n2️⃣ Attempting download using OpenAI documented pattern...")
    print(f"   📖 Reference: https://platform.openai.com/docs/api-reference/responses")
    print(f"   📖 OpenAI pattern: GET /v1/containers/{{container_id}}/files/{{file_id}}")
    
    # Azure OpenAI equivalent
    url = f"{AOAI_API_BASE}openai/v1/containers/{container_id}/files/{file_id}"
    
    token = token_provider()
    headers = {"Authorization": f"Bearer {token}"}
    params = {"api-version": "preview"}
    
    print(f"\n   🔗 Azure OpenAI Request:")
    print(f"      URL: {url}")
    print(f"      Headers: Authorization: Bearer [token]")
    print(f"      Params: {params}")
    
    try:
        response_req = requests.get(url, headers=headers, params=params, timeout=30)
        
        print(f"\n   📊 Response:")
        print(f"      Status Code: {response_req.status_code}")
        print(f"      Response Body: {response_req.text}")
        
        if response_req.status_code == 200:
            Path("./downloads").mkdir(exist_ok=True)
            file_path = Path("./downloads") / filename
            
            with open(file_path, 'wb') as f:
                f.write(response_req.content)
            
            print(f"      ✅ SUCCESS: File downloaded to {file_path}")
            return True
        else:
            print(f"      ❌ FAILURE: Cannot download container file")
            return False
            
    except Exception as e:
        print(f"      ❌ EXCEPTION: {str(e)}")
        return False

def print_summary():
    """
    Summary for PG
    """
    print(f"\n" + "=" * 60)
    print("📋 ISSUE SUMMARY FOR PG LOGGING")
    print("=" * 60)
    
    print(f"""
🔍 ISSUE: Files created by Code Interpreter (Responses API) cannot be downloaded

📖 EXPECTED: Files should be downloadable per OpenAI docs:
   GET /v1/containers/{{container_id}}/files/{{file_id}}

❌ ACTUAL: Azure OpenAI returns HTTP 404:
   GET {{endpoint}}/openai/v1/containers/{{container_id}}/files/{{file_id}}?api-version=preview

✅ CONFIRMED:
   - Files are created successfully
   - Container/File IDs provided in AnnotationContainerFileCitation
   - Authentication is working (gets 404, not 401/403)

❓ REQUEST: Is container file download:
   1. Not implemented in Azure OpenAI Responses API?
   2. Using different endpoint/method?
   3. Planned for future release?

📄 Details: API - {AOAI_API_VERSION}, Model - GPT-4.1-mini
""")

# Run the demonstration
success = demonstrate_container_file_access_issue()
print_summary()

🔍 Azure OpenAI - Issue with access to file in Code Interpreter's container

1️⃣ Using existing response: resp_68990aa90e4481908b55044faa2ce58e09409ad29506dd9e
   ✅ File found in annotations:
      Container ID: cntr_68990aaa13b88190810d4b9a5e1cc7b20727e1a613c30bf2
      File ID: cfile_68990aafbcd88190918923dc947b587b
      Filename: cfile_68990aafbcd88190918923dc947b587b.png

2️⃣ Attempting download using OpenAI documented pattern...
   📖 Reference: https://platform.openai.com/docs/api-reference/responses
   📖 OpenAI pattern: GET /v1/containers/{container_id}/files/{file_id}

   🔗 Azure OpenAI Request:
      URL: https://laziz-aoai-swc.openai.azure.com/openai/v1/containers/cntr_68990aaa13b88190810d4b9a5e1cc7b20727e1a613c30bf2/files/cfile_68990aafbcd88190918923dc947b587b
      Headers: Authorization: Bearer [token]
      Params: {'api-version': 'preview'}

   📊 Response:
      Status Code: 404
      Response Body: 
      ❌ FAILURE: Cannot download container file

📋 ISSUE SUMMARY FOR PG 